In [13]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import math
import xesmf as xe
from numpy import nan


from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import shapefile_utils as shp

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)
importlib.reload(shp)

import warnings
warnings.filterwarnings('ignore')

Set up model names, dates and CLIVAR_LE location

In [14]:
modnames=['CanESM2','CSIRO-Mk3-6-0','CESM1-CAM6','GFDL-CM3','MPI-ESM']
modnames_small=['canesm2','csiro_mk36','cesm','gfdl_cm3','mpi']
basedir="/glade/collections/cdg/data/CLIVAR_LE/"
ybegp=1979 ; monbegp=1 ; yendp=2014 ; monendp=12
ybegf=2070 ; monbegf=1 ; yendf=2099 ; monendf=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

nmonthsp=(yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf=(yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

pathout="/glade/scratch/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"
shpfile_usa = "/glade/scratch/islas/python/ecpaper2020/tmp/shpfiles/USA/gadm36_USA_1.shp"
shpfile_mex = "/glade/scratch/islas/python/ecpaper2020/tmp/shpfiles/Mexico/gadm36_MEX_0.shp"


Loop over models and members and output the DJF precipitation averaged over the three regions.

In [15]:
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

for imod in range(len(modnames)):
    moddirpr=basedir+modnames_small[imod]+"_lens/Amon/pr/"
    memfiles=!cd $moddirpr && ls *rcp85*.nc
    nmems=len(memfiles)
    reusewgt=False
    wgtfile="/glade/scratch/islas/python/tmp/wgtfile.nc"
    
    for imem in range(len(memfiles)):
        fname=memfiles[imem]
        print(fname)
        prp = read.read_sfc(moddirpr+fname, datebegp, dateendp)
        prf = read.read_sfc(moddirpr+fname, datebegf, dateendf)
        
        # check sizes
        if (prp.time.size != nmonthsp):
            print("something's wrong, "+str(nmonthsp)+
                 "expected, but got "+str(prp.time.size))
            sys.exit("failure at "+fname)
            
        if (prf.time.size != nmonthsf):
            print("something's wrong, "+str(monthsf)+
                  "expected, but got "+str(prf.time.size))
            sys.exit("failure at "+fname)
            
        prpdjf=cal.season_mean(prp,"pr",season="DJF")
        prfdjf=cal.season_mean(prf,"pr",season="DJF")
        
        prpdjf = prpdjf*86400.
        prfdjf = prfdjf*86400.
        
        if (imem == 0):
            prpgather = xr.DataArray(np.zeros([nmems, prpdjf.lat.size, prpdjf.lon.size]),
                                    coords=[np.arange(0,nmems), prpdjf['lat'], prpdjf['lon']],
                                    dims=['Member','lat','lon'], name='pr')
            prfgather = xr.DataArray(np.zeros([nmems, prfdjf.lat.size, prfdjf.lon.size]),
                                     coords=[np.arange(0,nmems), prfdjf['lat'], prfdjf['lon']],
                                     dims=['Member','lat','lon'], name='pr')
            
        prpgather[imem,:,:]=np.array(prpdjf)
        prfgather[imem,:,:]=np.array(prfdjf)
        
    prpgather = prpgather.rename("prpgather")
    prfgather = prfgather.rename("prfgather")
        
    regridder = xe.Regridder(prpgather, grid_out,'bilinear',
                            periodic=True, reuse_weights=reusewgt,
                            filename=wgtfile)
    prpinterp = regridder(prpgather)
    prfinterp = regridder(prfgather)
    
    maskwest = shp.maskgen(shpfile_usa, prpinterp, ['California','Oregon','Washington'])
    masksouth = shp.maskgen(shpfile_usa, prpinterp, 
                           ['Texas', 'New Mexico', 'Louisiana', 'Alabama', 'Mississippi', 'Oklahoma', 'Arkansas'])
    maskmex = shp.maskgen(shpfile_mex, prpinterp,
                         ['Mexico'])
    lat2d = np.repeat(np.expand_dims(maskmex.lat, axis=1), maskmex.lon.size, axis=1)
    maskmex = np.where(lat2d > 22, nan, maskmex)
    maskmex = xr.DataArray(maskmex, coords = grid_out.coords)
    
    prp_westmask = xr.DataArray(np.array(prpinterp) * np.array(maskwest), coords=prpinterp.coords)
    prp_southmask = xr.DataArray(np.array(prpinterp) * np.array(masksouth), coords=prpinterp.coords)
    prp_mexmask = xr.DataArray(np.array(prpinterp) * np.array(maskmex), coords=prpinterp.coords)
    
    prf_westmask = xr.DataArray(np.array(prfinterp) * np.array(maskwest), coords=prfinterp.coords)
    prf_southmask = xr.DataArray(np.array(prfinterp) * np.array(masksouth), coords=prfinterp.coords)
    prf_mexmask = xr.DataArray(np.array(prfinterp) * np.array(maskmex), coords=prfinterp.coords)
    
    prp_west = avg.cosweightlonlat(prp_westmask, 0, 360, -90, 90)
    prp_west = prp_west.rename("prp_west")
    prf_west = avg.cosweightlonlat(prf_westmask, 0, 360, -90, 90)
    prf_west = prf_west.rename("prf_west")
    prp_south = avg.cosweightlonlat(prp_southmask, 0, 360, -90, 90)
    prp_south = prp_south.rename("prp_south")
    prf_south = avg.cosweightlonlat(prf_southmask, 0, 360, -90, 90)
    prf_south = prf_south.rename("prf_south")
    prp_mex = avg.cosweightlonlat(prp_mexmask, 0, 360, -90, 90)
    prp_mex = prp_mex.rename("prp_mex")
    prf_mex = avg.cosweightlonlat(prf_mexmask, 0, 360, -90, 90)
    prf_mex = prf_mex.rename("prf_mex")
    
    prdata = xr.merge([prp_west, prf_west, prp_south, prf_south, prp_mex, prf_mex])
    
    prdata.to_netcdf(path=pathout+"/prdata_"+modnames[imod]+".nc")
    
    
    
    
    
    
    
    
            

pr_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r18i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r19i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r1i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r20i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r21i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r22i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r23i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r24i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r25i1p1_195001-210012.nc
pr_Amon_CanESM2